<a href="https://colab.research.google.com/github/Yazan-Daibes/Arabic-Toxic-Comment-Classification/blob/main/Final_16_1_22_Copy_of_IMP_Extract_word_embeddings_from_sentences_using_the_Transformers_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install transformers
!git clone https://github.com/aub-mind/arabert
!pip install -r arabert/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.3 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 17.37 MiB/s, done.
Resolving deltas: 100% (339/339), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 KB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji:

# Imports

In [ ]:
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess import ArabertPreprocessor

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelPropagation

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
#from sklearn.ensemble import RandomForestClassifier
#!pip uninstall xlrd -y
#!pip install --upgrade pandas
#!pip install 'xlrd == 1.2.0'
!pip install openpyxl 
import os
import io
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Initialize Model, Tokenizer and preprocessor

Available models are:
`aubmindlab/` +
```
bert-base-arabertv01
bert-base-arabert
bert-base-arabertv02
bert-base-arabertv2
bert-large-arabertv02
bert-large-arabertv2
araelectra-base-discriminator
araelectra-base-generator
aragpt2-base
aragpt2-medium
```

for `aragpt2-large` and `mega`, you need to use:
`from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel` instead of `AutoModel`

In [ ]:
#aubmindlab/bert-base-arabertv2
model_name = "aubmindlab/bert-large-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((


100%|██████████| 241M/241M [00:18<00:00, 12.8MiB/s]


[2023-02-21 06:34:16,836 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


Some weights of the model checkpoint at aubmindlab/bert-large-arabertv2 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Preprocessing the text before passing through the model

In [ ]:
#text= "شعرها جميل اليوم"
#text1= "عيونها جميلة اليوم"

#text_preprocessed = arabert_prep.preprocess(text1)                                
#print(text1)
#print("---------------------")
#print(text_preprocessed)

In [ ]:
data = pd.read_csv('shuffled-2.csv') # To read first 400 comments only
#data = pd.read_csv('Round1Class.csv')
data = data.rename( columns = {'comment':'Tweet','label':'Class'} , inplace= False )
data['Class'].value_counts()
#data["Class"].replace({"normal":"0","obscene":"1","identity hate":"2","physical appearance":"3","threat":"4"},inplace=True)

#data["Class"].replace({"normal":"0","abusive":"1","hate":"2","hateful":"2","abusive_hateful":"2","hateful_disrespectful":"2","obscene":"3"},inplace=True) # For four labels

#data["Class"].replace({"normal":"0","abusive":"1","hate":"2","hateful":"2","abusive_hateful":"2","hateful_disrespectful":"2","obscene":"1"},inplace=True) # For three labels



data["Class"].replace({"normal":"0","abusive":"1","hate":"1","hateful":"1","abusive_hateful":"1","hateful_disrespectful":"1","obscene":"1"},inplace=True) # For two labels


In [ ]:
test_data = data.sample(frac=0.3,random_state=200)
test_data.shape
data=data.drop(test_data.index)
data.shape,test_data.shape

((11045, 2), (4734, 2))

In [ ]:
data.rename(columns={'Tweet':'texts'}, inplace=True)
test_data.rename(columns={'Tweet':'texts'}, inplace=True)
data.rename(columns={'Class':'data_labels'}, inplace=True)
test_data.rename(columns={'Class':'data_labels'}, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train = data.texts.values
Y_train_labels = data.data_labels.values

X_test= test_data.texts.values
Y_test_labels = test_data.data_labels.values

In [ ]:
train_text_preprocessed = []
print(len(X_train))
for x in range(0,len(X_train)):
  train_text_preprocessed.append(arabert_prep.preprocess(X_train[x]))                                
  #print(X_train[x])
  #print("---------------------")
print(len(train_text_preprocessed))
print(train_text_preprocessed[6])

11045
11045
روح شوف معلم +ك جبران باسيل و سرق +ات +ه ب+ ال+ كهرباء يا حقود يا صاحب ال+ وجه ال+ بشع


In [ ]:
test_text_preprocessed = []
print(len(X_test))
for x in range(0,len(X_test)):
  test_text_preprocessed.append(arabert_prep.preprocess(X_test[x]))                                
  #print(X_test[x])
  #print("---------------------")
print(len(test_text_preprocessed))
print(test_text_preprocessed[0])

4734
4734
ال+ تيار ال+ عون +ي كتير و+ زرا من عند +هن عمبيغلطوا و+ اهم +هن ال+ وزير جبران باسيل يتصرف و+ كان كل لبنان تحت أمر جماع +ة ال+ ممانع +ة و+ ال+ منا …


In [ ]:
X_train.shape

(11045,)

In [ ]:
X_test.shape

(4734,)

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Converting the training text to tensors suitable for model input

In [ ]:
def get_embeddings(sentence):
  inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
  outputs = model(**inputs)
  embeddings = outputs['last_hidden_state']
  embeddings_text_only = outputs['last_hidden_state'][0][1:-1]
  embeddings_text_only = embeddings_text_only.detach().numpy()
  final_embeddings = np.mean(embeddings_text_only, axis=0)

  return final_embeddings

In [ ]:
# In case we want to save the embeddings so we don't wait long time for getting them from BERT
#f_train = open("train_embeddings_for_shuffled.txt","a")
#f_test = open("test_embeddings_for_shuffled.txt","a")


In [ ]:
#inputs_train is a list of dictionaries containing inputs_ids, attention_masks and token_type_ids as pytorch tensors

outputs_train = []
for x in range(0,len(train_text_preprocessed)):
  emb = get_embeddings(train_text_preprocessed[x])
  #f_train.write(str(emb))
  #f_train.write("\n")
  outputs_train.append(emb)



/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


Converting the testing text to tensors suitable for model input

In [ ]:
#inputs_test is a list of dictionaries containing inputs_ids, attention_masks and token_type_ids as pytorch tensors


outputs_test = []
for x in range(0,len(test_text_preprocessed)):
  emb = get_embeddings(test_text_preprocessed[x])
  #f_test.write(str(emb))
  #f_test.write("\n")
  outputs_test.append(emb)



In [ ]:
print(outputs_train[5])
print(outputs_test[5][0])
# To replace infinte values with finite numbers also replace zeros with numbers
import numpy
outputs_train = numpy.nan_to_num(outputs_train) 

outputs_test = numpy.nan_to_num(outputs_test) 



[-0.12608606 -0.2074103  -0.15840927 ...  0.17647308 -0.5579293
  0.55842346]
0.0928328


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import torch
import numpy as np



print(Y_train_labels)
print(len(Y_train_labels))
print(len(outputs_train))
print("===============================")
print(len(Y_test_labels))
print(len(outputs_test))
print("============================================")
#Y_train_labels_list = []
#for x in range(0,len(embeddings_train_text_only)):
#  Y_train_labels_list.append(list(Y_train_labels))
#print(Y_train_labels_list[4])
#print(len(Y_train_labels_list[319]))
#print(len(embeddings_train_text_only[319]))
#print(embeddings_train_text_only[319])

rf = RandomForestClassifier()

print("==============================")

print("==============================---------------------")
#t = embeddings_train_text_only[319][0].tolist()
#print(t)
print("==============================-------------------------")

#print(len(t))
#print(len(embeddings_train_text_only))
#print(Y_train_labels_list[0][0])
#print(len(Y_train_labels_list[0]))


#t = np.array(t)
#t = np.reshape(t, (-1,1))
#print(t)
'''
tensor2=[]
for i in range(0,len(outputs_train)):
  tensor1 = torch.tensor(outputs_train[i],requires_grad=True)
  tensor2.append(tensor1[0].detach().numpy())
print(tensor1[0].detach().numpy())
print("-----")
print(tensor2[319])
print(len(tensor2))
'''
rf.fit(outputs_train,Y_train_labels)
### For testing
'''
tensor3=[]
for i in range(0,len(outputs_test)):
  tensor4 = torch.tensor(outputs_test[i],requires_grad=True)
  tensor3.append(tensor4[0].detach().numpy())
print(tensor4[0].detach().numpy())
print("-----")
print("testing tensor:")
print(tensor3[19])
print("length of the testing:")
print(len(tensor3))
'''
print("Score:")
mean_accuracy = rf.score(outputs_test,Y_test_labels)
print("mean accuracy of Random Forest:")
print(mean_accuracy)


#print("Score for testing:")
#print(len(X_test))
#mean_accuracy = rf.score(X_test,Y_test_labels)
#print("mean accuracy of Random Forest:")
#print(mean_accuracy)


 # rf.score(tensor1.detach().numpy(),Y_train_labels_list[i]) 


['0' '1' '0' ... '0' '1' '0']
11045
11045
4734
4734
==============================---------------------
==============================-------------------------
Score:
mean accuracy of Random Forest:
0.9009294465568229


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

print(len(outputs_test[0]))
print(len(outputs_test))

y_pred_test_RF = rf.predict(outputs_test)
#precision, recall, f1_score, _ = precision_recall_fscore_support(Y_test_labels, y_pred_test_RF, average='binary') # This is only for one class
precision, recall, f1_score, _ = precision_recall_fscore_support(Y_test_labels, y_pred_test_RF, average='macro') # for multi class classification
cm = confusion_matrix(Y_test_labels, y_pred_test_RF) # confusion matrix
# Print the results
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1_score))
print("Confusion Matrix:")
#print(pd.DataFrame(cm, columns=np.unique(y), index=np.unique(y)))
print("Confusion Matrix: \n", cm)
label_sums = np.sum(cm, axis=1)
print("Label sums:", label_sums)
print(label_sums[0])
print(cm[0])
print(f'Normal:  {cm[0]} {label_sums[0]}')

print("=================================================================")
print(y_pred_test_RF)
print(len(y_pred_test_RF))

y_pred_test_proba_RF = rf.predict_proba(outputs_test)
print(y_pred_test_proba_RF)

print(X_test[0])
print(Y_test_labels[0])


1024
4734
Precision: 0.90
Recall: 0.90
F1-score: 0.90
Confusion Matrix:
Confusion Matrix: 
 [[2260  247]
 [ 222 2005]]
Label sums: [2507 2227]
2507
[2260  247]
Normal:  [2260  247] 2507
['0' '0' '1' ... '0' '0' '0']
4734
[[0.92       0.08      ]
 [0.82166667 0.17833333]
 [0.32666667 0.67333333]
 ...
 [0.505      0.495     ]
 [0.618      0.382     ]
 [0.585      0.415     ]]
التيار العوني كتير وزرا من عندهن عمبيغلطوا واهمهن الوزير جبران باسيل يتصرف وكان كل لبنان تحت امر جماعة الممانعة والمنا…
0


**FOR NAIVE BAYES**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Compute predicted probabilities
# clf = GaussianNB() # TRY THIS IT'S VERY IMPORTANT THEY SAY IT'S BETTER FOR CONTINUOUS DATA AND BERT IS CONTINOUS :
'''
MultinomialNB and GaussianNB are two different algorithms for performing Naive Bayes classification. The main difference between them is the way they estimate the class probabilities:

MultinomialNB is used for discrete count data (e.g., text data). It models the features using a multinomial distribution, which represents the number of times a word appears in a document.

GaussianNB is used for continuous data (e.g., real-valued features). It models the features using a Gaussian distribution, which represents the mean and standard deviation of the feature values.

In general, if the features are discrete and count-based, you should use MultinomialNB. If the features are continuous and real-valued, you should use GaussianNB. It's important to use the appropriate algorithm for your data, since Naive Bayes is sensitive to the distribution of the features.

'''

nb_model = MultinomialNB(alpha=1.8)
#nb_model = GaussianNB() # IT GAVE ACCRACUY = 0.514575411913815
# For solving negative values
scaler = MinMaxScaler()
X_train = scaler.fit_transform(outputs_train)
X_test_new = scaler.transform(outputs_test)

nb_model.fit(X_train, Y_train_labels)
print("Accuracy:")
mean_accuracy_NB = nb_model.score(X_test_new,Y_test_labels)
print(mean_accuracy_NB)
y_pred_test_NB = nb_model.predict(outputs_test)

precision, recall, f1_score, _ = precision_recall_fscore_support(Y_test_labels, y_pred_test_NB, average='macro')
# Calculate the confusion matrix

cm = confusion_matrix(Y_test_labels, y_pred_test_NB)
# Print the results
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1_score))
print("Confusion Matrix:")
#print(pd.DataFrame(cm, columns=np.unique(y), index=np.unique(y)))
print("Confusion Matrix: \n", cm)

#probs = nb_model.predict_proba(X_test)

# Evaluate the classifier
#evaluate_roc(probs, Y_test_labels)


Accuracy:
0.7594000844951415
Precision: 0.75
Recall: 0.63
F1-score: 0.57
Confusion Matrix:
Confusion Matrix: 
 [[ 690 1817]
 [  30 2197]]


**For support vector machine (SVM)**

In [ ]:
from sklearn import svm

svm_model = svm.SVC(kernel = 'rbf',probability=True)
svm_model.fit(outputs_train,Y_train_labels)
print("Accuracy:")
svm_model.score(outputs_test,Y_test_labels)



Accuracy:


0.8569919729615547

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

y_pred_test_SVM = svm_model.predict(outputs_test)
print(y_pred_test_SVM)
print(len(y_pred_test_SVM))
precision, recall, f1_score, _ = precision_recall_fscore_support(Y_test_labels, y_pred_test_SVM, average='macro')

# Calculate the confusion matrix
cm = confusion_matrix(Y_test_labels, y_pred_test_SVM)

# Print the results
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1_score))
print("Confusion Matrix:")
#print(pd.DataFrame(cm, columns=np.unique(y), index=np.unique(y)))
print("Confusion Matrix: \n", cm)



y_pred_test_proba_SVM = svm_model.predict_proba(outputs_test)
print(y_pred_test_proba_SVM)


['0' '0' '1' ... '0' '0' '0']
4734
Precision: 0.86
Recall: 0.86
F1-score: 0.86
Confusion Matrix:
Confusion Matrix: 
 [[2138  369]
 [ 308 1919]]
[[0.93848591 0.06151409]
 [0.66056829 0.33943171]
 [0.01249919 0.98750081]
 ...
 [0.69258595 0.30741405]
 [0.88397191 0.11602809]
 [0.7864443  0.2135557 ]]


In [ ]:
# AraGPT2 and AraELECTRA does not have a pooler layer, you just take the embedding of the last token for AraGPT2, and the first for araElectra
#pooled_vector = outputs['pooler_output']
#pooled_vector.shape # batch_size x emb_dim

In [ ]:
# For prediction:
def predict_sentence(sentences,the_model):
    label = the_model.predict(sentences)
    return label


In [ ]:
label = predict_sentence(outputs_test,svm_model)
print(label)
for x in range(0,len(label)):
  print(X_test[x])
  if label[x] == '0' :
    #print("Normal")
    print("Not Toxic - Normal")
  elif label[x] == '1':
    #print("Abusive")
    print("Toxic")
  elif label[x] == '2':
    print("Hate")
  elif label[x] == '3':
    print("Obscene")
#  print(label[x])

Streaming output truncated to the last 5000 lines.
حذف تغريدته لانه كلب من كلاب الترك شركسي الاصل او يوناني
Toxic
وكمان قالت له ما يأكل خنزير  بس معه بعلاقة غير شرعية. انا نفسي افهم معقول في ناس مش فاهمه الدين لهذه الدرجة. مشتيش… @url
Not Toxic - Normal
لا أحد يواجه العرب غير أسيادك في طهران و الشام 
Not Toxic - Normal
الجبناء امثالكم لا يستطيعون مواجهة الحقائق لأنكم زائفون وحججكم كبيوت العنكبوت خسئتم
Toxic
هذا لو كان عندهم عقل يفكروا به بول البعير جعلهم كالبهاءم
Toxic
مين انت يعني حتى عم تهدد انقبر احترم حالك يا نكرة
Toxic
حبيب فياض اعترف بمقابلة تلبفزيونبة بانو زعران حركة امل كلاب بيعضوا
Toxic
طز فيك و باللقاء تبعكن لقاء الأوباش و قطاع الطرق إن دعوت شيخ عقل الإقطاع الارسلاني هو تدخل بشؤون طائفة الم…
Toxic
هناك قضاة في لبنان لا يمكن شراؤهم
Not Toxic - Normal
والله زمان يا بن الهبلة ، يلا قوم معاد جلسة الكهرباء
Toxic
لا نخشاك ياسفاح الشام بحياة امك تخبرني بشو نتصر اسدك بس فاتح عيونك يا بهيم يلي عم دافع عنو هو يلي قتل ابنك وخيك وبيك وكل عايلتك هو يلي باع شرفك يا بلا شرف ما اقرفك
Toxic
ف